In [2]:
from module import connection
from module import get
from module import update
from module import search
from module import save
import pandas as pd

In [2]:
update.update_stock_info('000020_동화약품.csv')

20221105 20221117
20221106 20221117
20221107 20221117
20221108 20221117
20221109 20221117
20221110 20221117
20221111 20221117
20221112 20221117
20221113 20221117
20221114 20221117
20221115 20221117
20221116 20221117
20221117 20221117


,시간,시가,고가,저가,종가,전일대비,거래량,거래대금,누적체결매도수량,상장주식수,...,외국인현보유비율,수정주가일자,수정주가비율,기관순매수량,기관누적순매수량,등락주선,등락비율,예탁금,주식회전율,거래성립률
20221116,1530,9390,9390,9390,9390,0,1363,12800000,67745,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20221116,1520,9400,9400,9390,9390,0,335,3150000,67745,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20221116,1519,9390,9400,9390,9400,0,42,390000,67419,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20221116,1518,9390,9400,9380,9400,0,52,490000,67391,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20221116,1517,9390,9390,9380,9390,0,27,250000,67365,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20201027,905,16200,16300,16100,16250,0,20521,332280000,62820,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20201027,904,16400,16450,15900,16300,0,30445,493410000,50504,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20201027,903,16500,16500,16250,16450,0,18800,307520000,26098,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20201027,902,16650,16700,16300,16500,0,11727,194040000,17366,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0


In [4]:
test = pd.read_csv('../005930_삼성전자.csv')
test = test.drop(['시간', '시가'], axis=1)
test

,Unnamed: 0,고가,저가,종가,전일대비,거래량,거래대금,누적체결매도수량,상장주식수,시가총액,...,외국인현보유비율,수정주가일자,수정주가비율,기관순매수량,기관누적순매수량,등락주선,등락비율,예탁금,주식회전율,거래성립률
0,20221103,59200,59200,59200,0,1859760,110097790000,5944330,0,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
1,20221103,59700,59500,59700,0,193177,11510350000,5944330,0,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
2,20221103,59600,59500,59600,0,49046,2921250000,5893859,0,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
3,20221103,59600,59500,59600,0,20663,1230500000,5874904,0,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
4,20221103,59600,59500,59600,0,20021,1192690000,5864749,0,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190375,20201027,60000,59800,59800,0,450531,26982330000,730219,0,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
190376,20201027,60000,59900,59900,0,159408,9552240000,494948,0,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
190377,20201027,60000,59900,59900,0,41872,2509660000,372455,0,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
190378,20201027,60100,59900,60000,0,172825,10367550000,345917,0,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0


In [ ]:
from tqdm import tqdm
from pyarrow import csv
import os

for file in tqdm(os.listdir('../data/update/20221116/분봉/')):
        stock_info = csv.read_csv('../data/update/20221116/분봉/{0}'.format(file), encoding='utf-8 sig', index_col='Unnamed: 0').to_pandas()
        if stock_info.index[0] == 20221117:
            stock_info = stock_info.drop([20221117], axis=0)
            stock_info.to_csv('../data/update/20221116/분봉/{0}'.format(file), encoding='utf-8 sig')


In [ ]:
df = csv.read_csv('data.csv').to_pandas()